Pedersen Commitment
==================

## Pedersen commitment:

$Gen(1^\lambda) \rightarrow ck$

$$
g \leftarrow \mathbb{G}\\
h = g^x\\
ck:=(\mathbb{G}, p, g, h)
$$



$Com_{ck}(m) \rightarrow c$

$$
c:=(g^r, g^mh^r)
$$

In [1]:
from klefki.types.algebra.concrete import EllipticCurveCyclicSubgroupSecp256k1 as ECC
from klefki.types.algebra.concrete import EllipticCurveGroupSecp256k1 as Curve
from klefki.types.algebra.concrete import FiniteFieldCyclicSecp256k1 as CF
from klefki.types.algebra.concrete import FiniteFieldSecp256k1 as F
from klefki.types.algebra.utils import randfield

G = ECC.G
H = Curve.lift_x(F(1234567))


### $\Sigma$-protocol

Consider a commitment $A$ opening to m to be part of the statement. The prover computes a random commitment $B = Com_{ck}(m; s)$ and sends it to the veriﬁer, which answer with a random challenge $x$. The prover then sends opening information $z$ to the veriﬁer, which checks the commitment $A^x B$ opens to m using randomness $z$.

$s \leftarrow \mathbb{Z}_p$
$B=Com_{ck}(m;s)$
$$
P \rightarrow V: B
$$

In [2]:
m = randfield(CF)
r = randfield(CF)

A = G ** m + H ** r

In [3]:
s = randfield(CF)


B = G ** s * H ** r

$x \leftarrow \mathbb{Z}_p$

$$
P \leftarrow V: x
$$

In [4]:
e = randfield(CF)


$z = me + s; x = re + r$

$$
P \rightarrow V: z, x
$$

In [5]:
z = m*e + s
x = r*e + r

Accept $\iff$ $B \in \mathbf{G}, z \in \mathbb{Z}_p$

$$
Com_{zk}(z;x) = A^eB
$$

In [6]:
G ** z * H ** x == A ** e * B

True

## Implementation

In [7]:
from klefki.zkp.pedersen import PedersonCommitment

In [8]:
priv = randfield(CF)
m = randfield(CF)
r = randfield(CF)

s = randfield(CF)
e = randfield(CF)
P = PedersonCommitment(G, G@x, priv)

In [9]:
P.commit(m, r)

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::11958990026192754556979349466742807490168090815484412931302328244610424385993, FiniteFieldSecp256k1::45404608319901513240564353599061225492875918032255841859779440817091608666156),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::44639407474732705754559663789262830618598095859413148894038292395450241912473, FiniteFieldSecp256k1::24971696227430178475114129197755083286069855553277863584002517581475183181162))

In [10]:
P.challenge(e)

(FiniteFieldCyclicSecp256k1::112013845117635558483974699736784618787696771237130381528548919343662298885955,
 FiniteFieldCyclicSecp256k1::26353319631585903533084081766975777640350239364803789983631922830089971283489)

In [11]:
P.proof()

True

In [12]:
m1 = randfield(CF)


P.trapdoor(m1, x)

In [13]:
P.challenge(e)
P.proof()

True

## Ref:

* Eﬃcient Zero-Knowledge Proof Systems, Jonathan Bottle, ..., UCL
* Zero-Knowledge Proof and Cryptographic Commitment https://www.cs.purdue.edu/homes/ninghui/courses/555_Spring12/handouts/555_Spring12_topic23.pdf